In [ ]:
user='{your_email}@uber.com'
drogon_token = '{enter_your_drogon_token_here}'

hive_in='next.repeated_eats_order_prediction_features_global'

models = {
    'tm20210702-155235-DEKXIXRB-DSUGVX': {
        'predicted_label': '@prediction|predict',
        'predicted_prob_true': '@prediction|true',
        'project': 'next_eats_order_prediction'
    }
}
datacenter = 'PHX2' 
start_date = '2021-06-28'
end_date = '2021-06-28'
model_output = 'predict'
custom_prediction_filter = ""



spark_queue='/MappingServices/MapDataCalifornia/Basemap'
cluster='phx4/prod02'
"""
# spark_queue='/MappingServices/MapDataColorado/Mapstore'
# cluster='phx4/prod02'

spark_queue='/MappingServices/MapDataCalifornia/Basemap'
cluster='phx4/prod02'
"""

In [ ]:
# MA settings
jobs_config = {
    "is_prod": True
}
# spark_config = {
#     "spark.yarn.executor.memoryOverhead" :"16g",
#     "spark.ext.michelangelo.num_input_partitions": '1800'
# }

In [ ]:
!echo $VIRTUAL_ENV_DIR

In [ ]:
!python --version

In [ ]:
!python --version

In [ ]:
import michelangelo_python_client
print("michelangelo_python_client==" + michelangelo_python_client.__version__)

In [ ]:
!python -m pip install -U michelangelo_python_client

In [ ]:
!pip install queryrunner_client

In [ ]:
!install_package_python3.sh michelangelo-python-client==0.6.163

In [ ]:
!install_package_python3.sh queryrunner_client

In [ ]:
!install_package_python3.sh michelangelo_python_client

In [ ]:
import pandas as pd
from queryrunner_client import Client
import michelangelo.malib
from michelangelo.cmdline.lib.security_context import SecurityContext
from michelangelo.cmdline.lib.madlj import MadljConnector
from datetime import datetime
import time
# https://engdocs.uberinternal.com/michelangelo-python-client/library.html

In [ ]:
%env UBER_OWNER=$user
%env HADOOP_USER_NAME=$user

MadljConnector.set_security_context(
    SecurityContext(
        user_email=user,
        hdfs_delegation_token=drogon_token
    )
)

In [ ]:
jobs = {}
tables = {}
for model in models.keys():
    job = michelangelo.malib.spark_predict_run(
        project=models[model]['project'],
        model_ids=model,
        src_hive=hive_in,
        spark_queue=spark_queue,
        cluster=cluster,
        resource_manager=1,
        executors=200,
        executor_cores=4,
        executor_mem=150,
        driver_cores=4,
        driver_mem=64,
        datacenter=datacenter,
        publish_all_fields=False,
        owner=user,
        date_str="{start_date}:{end_date}".format(start_date=start_date, end_date=end_date),
        custom_prediction_filter=custom_prediction_filter,
        **jobs_config)
    job_id = job['Job ID']
    dst_hive = job['dst-hive']
    print("Model {model} has job id {job} and output of {dst}".format(
        model=model,
        job=job_id,
        dst=dst_hive
    ))
    jobs[model] = job_id
    tables[model]= dst_hive

In [ ]:
def update_info(jobs, info=None):
    out_info = {}
    for model in jobs.keys():
        if info is not None and info[model]['state'] in {'success', 'dead'}:
            out_info[model] = info[model]
        else:
            out_info[model] = michelangelo.malib.spark_predict_show(
                job_id=jobs[model],
                datacenter=datacenter,
                **jobs_config
            )
    return out_info

def all_finished(info):
    for response in info.values():
        if response['state'] not in {'success', 'dead'}:
            return False
    return True

In [ ]:
info = update_info(jobs, info=None)
info.keys()

In [ ]:
while not all_finished(info):
    for model in info.keys():
#     for model in ['tm20201001-025541-JWPRVUGZ']:
        print(datetime.now().time(), model, info[model]['state'], info[model]['sparkUiUrl'])
    time.sleep(30)
    # get one-off prediction job info
    info = update_info(jobs, info=info)
info

In [ ]:
query = """
CREATE TABLE next.eats_order_batch_prediction_{project}_{model} 
AS SELECT * from {table};

"""
for model in models.keys():
    print(
        query.format(
            table=tables[model],
            project=models[model]['project'].lower(),
            model=model.replace("-","_").lower()
        )
    )